In [64]:
import os
import torch
from tqdm import tqdm
import bitsandbytes as bnb
from copy import deepcopy

In [2]:
ckpt = torch.load("../checkpoints/llama2/Llama-2-7b/consolidated.00.pth",map_location='cpu')

In [133]:
wo_list = []
for key,val in ckpt.items():
    if key.endswith("wk.weight"):
        wo_list += [val.to("cuda", torch.float32)]

In [176]:
rank = 256
range_anchor = 2
# def low_rank_equivalent(base, target):
#     delta = target.clone().detach() - base.clone().detach()
#     u,s,v = torch.svd(delta.to(torch.float32))
#     k = rank
#     u_topk, s_topk, v_topk = u[:, :k], s[:k], v[:, :k]

#     lora_b = torch.mm(u_topk, torch.diag(s_topk.sqrt())).to(target.dtype)
#     lora_a = torch.mm(torch.diag(s_topk.sqrt()), v_topk.t()).to(target.dtype)
#     return lora_a, lora_b
# w_base = torch.nn.Parameter(wo_list[0].data.clone()).cuda()
# w_lora_list = []
# for ii in tqdm([kk for kk in range(1,32)]):
#     lora_a, lora_b = low_rank_equivalent(w_base.data, wo_list[ii])
#     lora1 = torch.nn.Parameter(lora_b.data.clone(),requires_grad=True)
#     lora2 = torch.nn.Parameter(lora_a.data.clone(),requires_grad=True)
#     w_lora_list +=[(lora1.cuda(), lora2.cuda())]
# lora1 = torch.nn.Parameter(torch.empty_like(wo_list[0][:,:rank]),requires_grad=True)
# lora2 = torch.nn.Parameter(torch.empty_like(wo_list[0][:rank,:]),requires_grad=True)
# torch.nn.init.xavier_normal_(lora1)
# torch.nn.init.xavier_normal_(lora2)
# w_lora_list = [(lora1.cuda(), lora2.cuda())] + w_lora_list


w_base = torch.nn.Parameter(torch.empty_like(wo_list[0])).cuda()
torch.nn.init.xavier_normal_(w_base)
w_lora_list = []
for ii in range(range_anchor):
    lora1 = torch.nn.Parameter(torch.empty_like(wo_list[0][:,:rank]),requires_grad=True)
    lora2 = torch.nn.Parameter(torch.empty_like(wo_list[0][:rank,:]),requires_grad=True)
    torch.nn.init.xavier_normal_(lora1)
    torch.nn.init.xavier_normal_(lora2)
    w_lora_list +=[(lora1.cuda(), lora2.cuda())]


weight = bnb.nn.Params4bit(
                wo_list[0].data.clone().cpu(), 
                requires_grad=False,
                quant_type='nf4',
)

In [179]:
weight.to("cuda")
weight_nf4 = bnb.functional.dequantize_4bit(weight, weight.quant_state)
((wo_list[0].data.clone()-weight_nf4)**2).mean()

tensor(3.2611e-06, device='cuda:0')

In [171]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD([w_lora_list[ii][0] for ii in range(range_anchor)]+[w_lora_list[ii][1] for ii in range(range_anchor)]+[w_base], lr=100000)

In [175]:
pbar = tqdm([ll for ll in range(500)], desc='Training', leave=True)
for epoch in pbar:
    optimizer.zero_grad()           # Zero the gradients
    loss = 0
    for idx, ww in enumerate(wo_list[:range_anchor]):
        w_approx = w_base + w_lora_list[idx][0] @ w_lora_list[idx][1]
        loss += criterion(w_approx, ww)  # Calculate loss
    # print(loss.item())
    loss.backward()
    pbar.set_description(f'Training - Epoch {epoch+1}, Loss: {loss.item():.2e}, w:{w_base[0,0].item():.2e} grad:{w_base.grad[0,0].item():.2e}')
    # print(w_base.grad[0,0])
    # print(w_base[0,0])
    optimizer.step()
print(loss)

Training - Epoch 500, Loss: 3.86e-05, w:2.99e-03 grad:-6.86e-12: 100%|██████████| 500/500 [00:02<00:00, 216.46it/s]

tensor(3.8600e-05, device='cuda:0', grad_fn=<AddBackward0>)


In [163]:
w_base.std(), w_base.mean()

(tensor(0.0144, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(3.7319e-06, device='cuda:0', grad_fn=<MeanBackward0>))

In [164]:
idx=0
(w_lora_list[idx][0] @ w_lora_list[idx][1]).std(),(w_lora_list[idx][0] @ w_lora_list[idx][1]).mean()

(tensor(0.0174, device='cuda:0', grad_fn=<StdBackward0>),
 tensor(3.2446e-06, device='cuda:0', grad_fn=<MeanBackward0>))

In [55]:
def low_rank_equivalent(base, target):
    delta = target.clone().detach() - base.clone().detach()
    u,s,v = torch.svd(delta.to(torch.float32))
    k = 1024
    u_topk, s_topk, v_topk = u[:, :k], s[:k], v[:, :k]

    lora_b = torch.mm(u_topk, torch.diag(s_topk.sqrt())).to(target.dtype)
    lora_a = torch.mm(torch.diag(s_topk.sqrt()), v_topk.t()).to(target.dtype)
    return base + lora_b@lora_a
delta = 0
for ww in tqdm(wo_list[1:]):
    new_ww = low_rank_equivalent(wo_list[0],ww)
    delta += ((ww-new_ww)**2).mean()
print(delta)

100%|██████████| 31/31 [00:59<00:00,  1.92s/it]

tensor(0.0039, device='cuda:0')


In [210]:
ckpt_new = deepcopy(ckpt)

In [211]:
for ending_name in ["wq.weight","wk.weight","wv.weight","wo.weight"]:
    wo_list = []
    w_name_list = []
    for key,val in ckpt.items():
        if key.endswith(ending_name):
            wo_list += [val.to("cuda", torch.float32)]
            w_name_list += [key]

    rank = 512
    range_anchor = 2
    print(f"(In a group) full params:{4096*4096*range_anchor}, retained params:{4096*rank*2*range_anchor+4096}, reduced:{(4096*rank*2*range_anchor+4096)/(4096*4096*range_anchor):.3f}")

    for group_idx in range(0,32,range_anchor):
        w_base = torch.nn.Parameter(torch.empty_like(wo_list[group_idx])).cuda()
        torch.nn.init.xavier_normal_(w_base)
        w_lora_list = []
        for ii in range(range_anchor):
            lora1 = torch.nn.Parameter(torch.empty_like(wo_list[0][:,:rank]),requires_grad=True)
            lora2 = torch.nn.Parameter(torch.empty_like(wo_list[0][:rank,:]),requires_grad=True)
            torch.nn.init.xavier_normal_(lora1)
            torch.nn.init.xavier_normal_(lora2)
            w_lora_list +=[(lora1.cuda(), lora2.cuda())]

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.SGD([w_lora_list[ii][0] for ii in range(range_anchor)]+[w_lora_list[ii][1] for ii in range(range_anchor)]+[w_base], lr=500000)

        pbar = tqdm([ll for ll in range(1000)], desc=f'Training group_idx={group_idx}', leave=True)
        for epoch in pbar:
            optimizer.zero_grad()           # Zero the gradients
            loss = 0
            names = w_name_list[group_idx:group_idx+range_anchor]
            for idx, ww in enumerate(wo_list[group_idx:group_idx+range_anchor]):
                w_approx = w_base + w_lora_list[idx][0] @ w_lora_list[idx][1]
                loss += criterion(w_approx, ww)  # Calculate loss
            loss.backward()
            pbar.set_description(f'Training - Epoch {epoch+1}, Loss: {loss.item():.2e}, w:{w_base[0,0].item():.2e} grad:{w_base.grad[0,0].item():.2e}')
            optimizer.step()
        for layer_idx, name in enumerate(names):
            ckpt_new[name] = w_base.to(ckpt_new[name].dtype)
            ckpt_new[name.replace("weight","lora_a.weight")] = w_lora_list[layer_idx][1]
            ckpt_new[name.replace("weight","lora_b.weight")] = w_lora_list[layer_idx][0]
        print(group_idx,loss, names)
        print("-"*40)

(In a group) full params:33554432, retained params:8392704, reduced:0.250


Training - Epoch 1000, Loss: 1.04e-05, w:4.93e-03 grad:9.92e-12: 100%|██████████| 1000/1000 [00:07<00:00, 131.09it/s]


0 tensor(1.0443e-05, device='cuda:0', grad_fn=<AddBackward0>) ['layers.0.attention.wq.weight', 'layers.1.attention.wq.weight']
----------------------------------------


Training - Epoch 1000, Loss: 1.16e-04, w:-8.26e-03 grad:-1.39e-12: 100%|██████████| 1000/1000 [00:07<00:00, 131.77it/s]


2 tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) ['layers.2.attention.wq.weight', 'layers.3.attention.wq.weight']
----------------------------------------


...

26 tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) ['layers.26.attention.wo.weight', 'layers.27.attention.wo.weight']
----------------------------------------


Training - Epoch 1000, Loss: 1.47e-04, w:1.18e-02 grad:-5.97e-12: 100%|██████████| 1000/1000 [00:07<00:00, 131.68it/s]


28 tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) ['layers.28.attention.wo.weight', 'layers.29.attention.wo.weight']
----------------------------------------


Training - Epoch 1000, Loss: 1.43e-04, w:1.14e-02 grad:-2.65e-12: 100%|██████████| 1000/1000 [00:07<00:00, 131.23it/s]

30 tensor(0.0001, device='cuda:0', grad_fn=<AddBackward0>) ['layers.30.attention.wo.weight', 'layers.31.attention.wo.weight']
----------------------------------------


In [212]:
ckpt_new.keys()

dict_keys(['tok_embeddings.weight', 'norm.weight', 'output.weight', 'layers.0.attention.wq.weight', 'layers.0.attention.wk.weight', 'layers.0.attention.wv.weight', 'layers.0.attention.wo.weight', 'layers.0.feed_forward.w1.weight', 'layers.0.feed_forward.w2.weight', 'layers.0.feed_forward.w3.weight', 'layers.0.attention_norm.weight', 'layers.0.ffn_norm.weight', 'layers.1.attention.wq.weight', 'layers.1.attention.wk.weight', 'layers.1.attention.wv.weight', 'layers.1.attention.wo.weight', 'layers.1.feed_forward.w1.weight', 'layers.1.feed_forward.w2.weight', 'layers.1.feed_forward.w3.weight', 'layers.1.attention_norm.weight', 'layers.1.ffn_norm.weight', 'layers.2.attention.wq.weight', 'layers.2.attention.wk.weight', 'layers.2.attention.wv.weight', 'layers.2.attention.wo.weight', 'layers.2.feed_forward.w1.weight', 'layers.2.feed_forward.w2.weight', 'layers.2.feed_forward.w3.weight', 'layers.2.attention_norm.weight', 'layers.2.ffn_norm.weight', 'layers.3.attention.wq.weight', 'layers.3.atten

In [213]:
torch.save(ckpt_new, "../checkpoints/effiLLaMA2/consolidated.00.pth")

In [214]:
print(ckpt_new["layers.30.attention.wq.weight"]==ckpt_new["layers.31.attention.wq.weight"])
print(ckpt_new["layers.10.attention.wk.weight"]==ckpt_new["layers.11.attention.wk.weight"])
print(ckpt_new["layers.2.attention.wv.weight"]==ckpt_new["layers.3.attention.wv.weight"])

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]], device='cuda:0')
tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]], device='cuda:0')
tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, Tr

In [205]:
ckpt_new["layers.31.attention.wq.lora_b.weight"].shape, ckpt_new["layers.31.attention.wq.lora_a.weight"].shape, 

(torch.Size([4096, 512]), torch.Size([512, 4096]))

In [209]:
ckpt_new["layers.3.attention.wk.weight"]==ckpt_new["layers.2.attention.wk.weight"]

tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [ ]:
model.llma.layers[31].attention.wq.weight
model.llma.layers[2].attention.wk.weight

In [ ]:
for ending_name in ["feed_forward.w1"]:
    wo_list = []
    w_name_list = []
    for key,val in ckpt.items():
        if key.endswith(ending_name):
            wo_list += [val.to("cuda", torch.float32)]
            w_name_list += [key]

    rank = 512
    range_anchor = 2
    print(f"(In a group) full params:{4096*4096*range_anchor}, retained params:{4096*rank*2*range_anchor+4096}, reduced:{(4096*rank*2*range_anchor+4096)/(4096*4096*range_anchor):.3f}")

    for group_idx in range(0,32,range_anchor):
        w_base = torch.nn.Parameter(torch.empty_like(wo_list[group_idx])).cuda()
        torch.nn.init.xavier_normal_(w_base)
        w_lora_list = []
        for ii in range(range_anchor):
            lora1 = torch.nn.Parameter(torch.empty_like(wo_list[0][:,:rank]),requires_grad=True)
            lora2 = torch.nn.Parameter(torch.empty_like(wo_list[0][:rank,:]),requires_grad=True)
            torch.nn.init.xavier_normal_(lora1)
            torch.nn.init.xavier_normal_(lora2)
            w_lora_list +=[(lora1.cuda(), lora2.cuda())]

        criterion = torch.nn.MSELoss()
        optimizer = torch.optim.SGD([w_lora_list[ii][0] for ii in range(range_anchor)]+[w_lora_list[ii][1] for ii in range(range_anchor)]+[w_base], lr=500000)

        pbar = tqdm([ll for ll in range(1000)], desc=f'Training group_idx={group_idx}', leave=True)
        for epoch in pbar:
            optimizer.zero_grad()           # Zero the gradients
            loss = 0
            names = w_name_list[group_idx:group_idx+range_anchor]
            for idx, ww in enumerate(wo_list[group_idx:group_idx+range_anchor]):
                w_approx = w_base + w_lora_list[idx][0] @ w_lora_list[idx][1]
                loss += criterion(w_approx, ww)  # Calculate loss
            loss.backward()
            pbar.set_description(f'Training - Epoch {epoch+1}, Loss: {loss.item():.2e}, w:{w_base[0,0].item():.2e} grad:{w_base.grad[0,0].item():.2e}')
            optimizer.step()
        for layer_idx, name in enumerate(names):
            ckpt_new[name] = w_base.to(ckpt_new[name].dtype)
            ckpt_new[name.replace("weight","lora_a.weight")] = w_lora_list[layer_idx][1]
            ckpt_new[name.replace("weight","lora_b.weight")] = w_lora_list[layer_idx][0]
        print(group_idx,loss, names)
        print("-"*40)